<a href="https://colab.research.google.com/github/yoshihikom/thu_Details_of_Management_Information_BII_Lesson12/blob/main/hara2021_chap6_12_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.12 テキスト分析(3) word2vec モデル
word2vecでは、ニューラルネットワークを用い、ある語と周辺語の対応関係を示す重み付け行列を作成することで、ある特定の語群の中の語の特性をベクトル化することが可能になります。語句を数値で示すことが可能になるため、例えば語と語の類似度について、数値にて示すことが可能になります。

まず、前節の前節のコラムにて利用したDBpedia.org由来の経営学者リストについて、MeCabを用いた分かち書きを行います。

In [2]:
!pip install SPARQLWrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.5 MB/s eta 0:00:00


In [3]:
# DBpedia.orgから日本の経営学者の情報を取得する

from SPARQLWrapper import SPARQLWrapper
import codecs

#
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#日本の経済学者の情報をまとめて取得する
sparql2 = SPARQLWrapper(endpoint='http://ja.dbpedia.org/sparql', returnFormat='json')
sparql2.setQuery("""
PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>

select distinct ?name ?abstract where {
    ?company <http://dbpedia.org/ontology/wikiPageWikiLink> <http://ja.dbpedia.org/resource/Category:日本の経営学者> .
    ?company rdfs:label ?name .
    ?company <http://dbpedia.org/ontology/abstract> ?abstract .
}
""")
results = []
results2 = sparql2.query().convert()

# このままresult2をprilltするとjson形式のデータで出力されるため、nameおよび bstractのvalueのみ取り出すことにしましょう。
# 結果を整形
for result in results2["results"]["bindings"]:
#    print("経済",";",result["name"]["value"],";",result["abstract"]["value"])
    results.append(result["abstract"]["value"])
# これにより,Wikipedia(DBPedia.org)に掲載された経済学者の名前およびその概要に関する情報を一括して取得することが出来ました
#このリストをtextに変換。リスト要素の区切りは改行とする。
text = '\n'.join(results)

f = codecs.open('経営学者リスト_dbpedia.txt', 'w', 'utf8')
f.write(text)
f.close()

In [5]:
!pip install mecab-python3
!pip install unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=f3909aad7c5ccc7837a05e156d88bab0465e02868535d25599eaeaac4daaad30
  Stored in directory: /root/.cache/pip/wheels/89/e8/68/f9ac36b8cc6c8b3c96888cd57434abed96595d444f42243853
Successfully built unidic-lite


In [6]:
# 取得した経営学者の情報を分かち書きにする

import MeCab
import codecs

tagger = MeCab.Tagger('-O wakati')

fi = codecs.open('経営学者リスト_dbpedia.txt', 'r', 'utf8')
fo = codecs.open('経営学者リスト_dbpedia_wakachi.txt', 'w', 'utf8')

line = fi.readline()

while line:
    result = tagger.parse("　"+line)
    fo.write(result[1:])
    line = fi.readline()

fi.close()
fo.close()

In [7]:
#word2vec モデルを構築する
#分かち書きしたファイルを読み込み，学習モデルを作成する

# gensimのインストールが必要
# anaconda prompt で以下のコマンドを実行
# conda install gensim
# 途中で　Proceed ([y]/n)? 　に"y"を入力
#

from gensim.models import word2vec

sentences = word2vec.LineSentence('経営学者リスト_dbpedia_wakachi.txt')
model = word2vec.Word2Vec(sentences,
                          sg=1,         #0: CBOW, 1: skip-gram
                          vector_size=300,     # ベクトルの次元数　sizeから変更
                          window=10,    # 入力単語からの最大距離
                          min_count=5,  # 単語の出現回数でフィルタリング
                          hs =1,
                          negative=0,
                          epochs=100)   # 学習を行う回数　iterから変更

model.save('./econ_word2vec')

In [8]:
model = word2vec.Word2Vec.load('./econ_word2vec')

word_vectors = model.wv.index_to_key   # どのようなタームがベクトル化されたを見る
print(word_vectors)

['、', '。', '大学', '年', '経営', '学', '（', 'の', '）', 'は', '教授', '日本', '者', '-', '・', '研究', '月', '日', '部', '経済', 'を', 'に', '名誉', '博士', '論', '院', 'と', '専門', 'で', 'し', 'た', 'て', '科', '学会', '商学', '東京', '長', '「', '」', '会長', '企業', '管理', '理事', 'など', '元', '県', '出身', '市', '賞', '会計', '一橋', '学位', '組織', '神戸', '情報', 'が', '役', '同', '法人', 'マーケティング', '学長', '助', '学院', '社会', '3', '国際', '史', '産業', '会', '大阪', 'も', '生まれ', '論文', '戦略', '取締', '2', '5', '所', '課程', '講師', '1', '歴任', ')', '早稲田', '家', '11', '会社', 'いる', '8', '中', '(', '等', '6', '委員', '10', 'ビジネス', '12', 'や', '第', '7', '科学', 'マネジメント', '4', '副', '義塾', '的', '株式', '昭和', '慶應', '受賞', '商科', '9', '政策', '学校', 'イノベーション', '県立', '卒', '静岡', '客員', '明治', '員', '務め', '経', '学者', 'システム', '京都', 'お', 'する', '学園', '卒業', '協会', 'か', '専攻', '』', '『', 'さ', '館', '法政', '工学', '文化', '中央', '定年', '士', '.', 'ある', '代表', '総合', '学術', '技術', '准', 'センター', '人', '小', '学科', '開発', '行動', '工業', '立', 'る', '取得', '北海道', '修了', 'から', 'おけ', '財団', '立命', '顧問', '分野', 'その', '専任', '関西', '都', 'だ', 'スクール', 'な', 

In [9]:
# 特定の語と似通った傾向を持つ単語リストを抽出
similar_words = model.wv.most_similar(positive=["イノベーション"], topn=17)
print(*[" ".join([v, str("{:.2f}".format(s))]) for v, s in similar_words], sep="\n")

ユーザー 0.35
製品 0.31
研究 0.26
von 0.26
共著 0.25
サービス 0.25
/ 0.24
Hippel 0.24
Innovation 0.24
発表 0.24
小川 0.23
技術 0.23
附属 0.23
貢献 0.22
of 0.22
特に 0.22
価値 0.22


In [11]:
# 言葉のベクトル表現の中身を確認する
word_vector = model.wv["一橋"]
print(word_vector)

[-0.14838135  0.05938765 -0.02387092 -0.05669586  0.15053408 -0.14241107
  0.07565844 -0.3653121  -0.24956366 -0.02299739  0.05723536 -0.10332806
  0.10059394  0.2675152  -0.01865878 -0.07095515 -0.02494612 -0.00432791
  0.24159752  0.28198087 -0.06049833  0.20088033  0.13445264 -0.00157742
  0.00429132  0.37750155  0.20809025 -0.19312744 -0.2508946  -0.19506535
  0.27530894 -0.06652881 -0.0011755  -0.2909467  -0.06195265  0.08299558
  0.09004067  0.05435052 -0.12594607 -0.05823507 -0.10125442  0.08140841
  0.25142744  0.3062659   0.06186137 -0.04014807  0.3278429  -0.19216947
 -0.30154288 -0.01571025  0.09410752  0.00078954  0.03400939 -0.12537715
  0.252637    0.01805845 -0.40589413  0.01202809  0.3265255  -0.3085587
 -0.15225373  0.03864082 -0.09043116 -0.06806701 -0.12504134  0.10311908
  0.1343988  -0.06611704 -0.14577924 -0.15765533  0.27728888  0.05490914
  0.12411046 -0.02383462 -0.3067358   0.00667785 -0.13970655  0.29584524
 -0.02320164  0.1309066  -0.41133022 -0.01695732 -0.

In [14]:
# 特定の語と語の間で似たような傾向を有するタームを取り出す
results = model.wv.most_similar(positive=[u'一橋'],negative=[u'東京'], topn=7)
for result in results:
    print(result[0])

太田
門下
補佐
なっ
Innovation
ロンドン
元


In [18]:
# 語句間の類似性について抽出
import pprint
pprint.pprint(model.wv.similarity('早稲田', '東京'))
pprint.pprint(model.wv.similarity('早稲田', '九州'))
pprint.pprint(model.wv.similarity('早稲田', 'イノベーション'))
pprint.pprint(model.wv.similarity('早稲田', '学会'))
pprint.pprint(model.wv.similarity('早稲田', '賞'))
pprint.pprint(model.wv.similarity('早稲田', '慶應'))

0.07976403
0.09648083
-0.117024854
0.05877335
0.009412604
0.11790089
